In [1]:
# Add the directory to sys.path
import sys
sys.path.append('.')

# Now you can import the module
from utils.tle_processing import *


In [2]:
import numpy as np
import spacetrack.operators as op
from datetime import datetime,timedelta
import pandas as pd
from spacetrack import SpaceTrackClient
from io import StringIO  # Import StringIO
import time
import matplotlib.pyplot as plt
import tqdm

In [17]:
# Model training
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge, HuberRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor


In [4]:
# Save model
import joblib
import os

# Classical ML Models


In [39]:
resample_tle_df = pd.read_csv('sample_dataset/sample_daily_records_data.csv')
resample_tle_df.set_index('Epoch', inplace=True)
resample_tle_df.head()

,First Derivative Mean Motion,Inclination (degrees),Right Ascension of the Ascending Node (degrees),Argument of Perigee (degrees),Mean Anomaly (degrees),Eccentricity,Mean Motion (revolutions per day),Revolution Number at Epoch
Epoch,,,,,,,,
2022-12-31,0.000157,51.644800,77.042350,208.400550,320.463100,0.000525,15.497978,37584.500000
2023-01-01,0.000166,51.644567,74.151133,211.103567,277.293067,0.000519,15.498221,37593.666667
2023-01-02,0.000175,51.644680,69.554840,215.693640,221.644740,0.000511,15.498584,37608.000000
2023-01-03,0.000162,51.644860,64.713420,219.864140,202.583880,0.000499,15.498874,37623.200000
2023-01-04,0.000142,51.644740,59.072260,225.670000,219.529780,0.000497,15.499123,37640.800000


In [7]:
resample_tle_df.columns

Index(['First Derivative Mean Motion', 'Inclination (degrees)',
       'Right Ascension of the Ascending Node (degrees)',
       'Argument of Perigee (degrees)', 'Mean Anomaly (degrees)',
       'Eccentricity', 'Mean Motion (revolutions per day)',
       'Revolution Number at Epoch'],
      dtype='object')

# Data Preparation

In [18]:
# --- 1. Prepare Sample Data ---
data_for_training = resample_tle_df.copy()

## First differencing

In [19]:


data_for_training['Revolution Number at Epoch_Diff'] =  data_for_training['Revolution Number at Epoch'].diff()

# data_for_training.head()


## Cyclical features

In [32]:
def encode_cyclical_feature(df: pd.DataFrame, column_name: str, period: float = 360.0) -> pd.DataFrame:
    """
    Encodes a cyclical feature (like degrees, hours, or months) using
    sine and cosine projection.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the cyclical feature column (in degrees).
        period (float): The length of the cycle (e.g., 360 for degrees, 24 for hours).
                        Defaults to 360.0.

    Returns:
        pd.DataFrame: A new DataFrame with two added columns:
                      <column_name>_sin and <column_name>_cos.
    """
    print(f"--- Encoding '{column_name}' with period {period} ---")

    # 1. Conversion to Radians: Necessary for NumPy's sin/cos functions.
    # The formula converts the value into a proportion of the cycle (0 to 2*pi).
    angle_rad = 2 * np.pi * df[column_name] / period

    # 2. Sine and Cosine Projection
    sin_col = f'{column_name}_sin'
    cos_col = f'{column_name}_cos'

    df[sin_col] = np.sin(angle_rad)
    df[cos_col] = np.cos(angle_rad)

    # Optional: Drop the original column if you don't need it.
    # df = df.drop(columns=[column_name])

    print(f"Created columns: '{sin_col}' and '{cos_col}'")
    return df

def decode_cyclical_feature(df: pd.DataFrame, sin_col: str, cos_col: str, period: float = 360.0) -> pd.DataFrame:
    """
    Reconstructs the original cyclical feature value (in degrees) from its
    sine and cosine components.

    Args:
        df (pd.DataFrame): The input DataFrame containing sine and cosine components.
        sin_col (str): The name of the sine component column.
        cos_col (str): The name of the cosine component column.
        period (float): The length of the cycle (e.g., 360 for degrees, 24 for hours).
                        Defaults to 360.0.

    Returns:
        pd.DataFrame: The DataFrame with a new reconstructed column.
    """
    print(f"\n--- Decoding from '{sin_col}' and '{cos_col}' ---")
    reconstructed_col = f'{sin_col.split("_sin")[0]}_reconstructed'

    # 1. Use arctan2 to get the angle in radians (range: [-pi, pi]).
    # arctan2 is crucial as it correctly handles all four quadrants.
    angle_rad = np.arctan2(df[sin_col], df[cos_col])

    # 2. Scale back to the original period (e.g., [0, 360]).
    # Formula: (angle_rad / (2 * pi)) * period
    angle_normalized = angle_rad * (period / (2 * np.pi))

    # 3. Handle the cyclical nature to ensure the result is non-negative (e.g., [0, 360)).
    # The decoded value might be negative (e.g., -90 degrees). We shift it to the
    # equivalent positive value (e.g., 270 degrees).
    df[reconstructed_col] = np.where(angle_normalized < 0,
                                     angle_normalized + period,
                                     angle_normalized)

    print(f"Reconstructed column: '{reconstructed_col}'")
    return df

In [33]:
def encode_cyclical_feature(df: pd.DataFrame, column_name: str, period: float = 360.0) -> pd.DataFrame:
    """
    Encodes a cyclical feature (like degrees, hours, or months) using
    sine and cosine projection.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the cyclical feature column (in degrees).
        period (float): The length of the cycle (e.g., 360 for degrees, 24 for hours).
                        Defaults to 360.0.

    Returns:
        pd.DataFrame: A new DataFrame with two added columns:
                      <column_name>_sin and <column_name>_cos.
    """
    print(f"--- Encoding '{column_name}' with period {period} ---")

    # 1. Conversion to Radians: Necessary for NumPy's sin/cos functions.
    # The formula converts the value into a proportion of the cycle (0 to 2*pi).
    angle_rad = 2 * np.pi * df[column_name] / period

    # 2. Sine and Cosine Projection
    sin_col = f'{column_name}_sin'
    cos_col = f'{column_name}_cos'

    df[sin_col] = np.sin(angle_rad)
    df[cos_col] = np.cos(angle_rad)

    # Optional: Drop the original column if you don't need it.
    # df = df.drop(columns=[column_name])

    print(f"Created columns: '{sin_col}' and '{cos_col}'")
    return df

def decode_cyclical_feature(df: pd.DataFrame, sin_col: str, cos_col: str, period: float = 360.0) -> pd.DataFrame:
    """
    Reconstructs the original cyclical feature value (in degrees) from its
    sine and cosine components.

    Args:
        df (pd.DataFrame): The input DataFrame containing sine and cosine components.
        sin_col (str): The name of the sine component column.
        cos_col (str): The name of the cosine component column.
        period (float): The length of the cycle (e.g., 360 for degrees, 24 for hours).
                        Defaults to 360.0.

    Returns:
        pd.DataFrame: The DataFrame with a new reconstructed column.
    """
    print(f"\n--- Decoding from '{sin_col}' and '{cos_col}' ---")
    reconstructed_col = f'{sin_col.split("_sin")[0]}_reconstructed'

    # 1. Use arctan2 to get the angle in radians (range: [-pi, pi]).
    # arctan2 is crucial as it correctly handles all four quadrants.
    angle_rad = np.arctan2(df[sin_col], df[cos_col])

    # 2. Scale back to the original period (e.g., [0, 360]).
    # Formula: (angle_rad / (2 * pi)) * period
    angle_normalized = angle_rad * (period / (2 * np.pi))

    # 3. Handle the cyclical nature to ensure the result is non-negative (e.g., [0, 360)).
    # The decoded value might be negative (e.g., -90 degrees). We shift it to the
    # equivalent positive value (e.g., 270 degrees).
    df[reconstructed_col] = np.where(angle_normalized < 0,
                                     angle_normalized + period,
                                     angle_normalized)

    print(f"Reconstructed column: '{reconstructed_col}'")
    return df

In [34]:
# ['Right Ascension of the Ascending Node (degrees)','Argument of Perigee (degrees)', 'Mean Anomaly (degrees)']
CYCLE_PERIOD = 360.0
FEATURE_COL = 'Right Ascension of the Ascending Node (degrees)'
data_for_training = encode_cyclical_feature(data_for_training, FEATURE_COL, CYCLE_PERIOD)

FEATURE_COL = 'Argument of Perigee (degrees)'
data_for_training = encode_cyclical_feature(data_for_training, FEATURE_COL, CYCLE_PERIOD)

FEATURE_COL = 'Mean Anomaly (degrees)'
data_for_training = encode_cyclical_feature(data_for_training, FEATURE_COL, CYCLE_PERIOD)


data_for_training.head()

--- Encoding 'Right Ascension of the Ascending Node (degrees)' with period 360.0 ---
Created columns: 'Right Ascension of the Ascending Node (degrees)_sin' and 'Right Ascension of the Ascending Node (degrees)_cos'
--- Encoding 'Argument of Perigee (degrees)' with period 360.0 ---
Created columns: 'Argument of Perigee (degrees)_sin' and 'Argument of Perigee (degrees)_cos'
--- Encoding 'Mean Anomaly (degrees)' with period 360.0 ---
Created columns: 'Mean Anomaly (degrees)_sin' and 'Mean Anomaly (degrees)_cos'


,First Derivative Mean Motion,Inclination (degrees),Right Ascension of the Ascending Node (degrees),Argument of Perigee (degrees),Mean Anomaly (degrees),Eccentricity,Mean Motion (revolutions per day),Revolution Number at Epoch,Revolution Number at Epoch_Diff,Right Ascension of the Ascending Node (degrees)_sin,Right Ascension of the Ascending Node (degrees)_cos,Argument of Perigee (degrees)_sin,Argument of Perigee (degrees)_cos,Mean Anomaly (degrees)_sin,Mean Anomaly (degrees)_cos
Epoch,,,,,,,,,,,,,,,
2023-01-01,0.000166,51.644567,74.151133,211.103567,277.293067,0.000519,15.498221,37593.666667,9.166667,0.961985,0.273101,-0.516587,-0.856235,-0.991910,0.126945
2023-01-02,0.000175,51.644680,69.554840,215.693640,221.644740,0.000511,15.498584,37608.000000,14.333333,0.937007,0.349311,-0.583451,-0.812148,-0.664510,-0.747279
2023-01-03,0.000162,51.644860,64.713420,219.864140,202.583880,0.000499,15.498874,37623.200000,15.200000,0.904183,0.427146,-0.640969,-0.767566,-0.384036,-0.923318
2023-01-04,0.000142,51.644740,59.072260,225.670000,219.529780,0.000497,15.499123,37640.800000,17.600000,0.857816,0.513957,-0.715327,-0.698790,-0.636479,-0.771294
2023-01-05,0.000139,51.644820,54.697760,228.970120,250.770840,0.000490,15.499368,37654.600000,13.800000,0.816115,0.577890,-0.754367,-0.656453,-0.944209,-0.329347


In [35]:
data_for_training.dropna(inplace=True)

# Column selection for model training

In [36]:
gt_features = ['First Derivative Mean Motion', 'Inclination (degrees)',
       'Right Ascension of the Ascending Node (degrees)',
       'Argument of Perigee (degrees)', 'Mean Anomaly (degrees)',
       'Eccentricity', 'Mean Motion (revolutions per day)',
       'Revolution Number at Epoch']

In [37]:
original_features = ['First Derivative Mean Motion', 'Inclination (degrees)','Eccentricity','Mean Motion (revolutions per day)']

diff_features = ['Revolution Number at Epoch_Diff']

cyclical_features = ['Right Ascension of the Ascending Node (degrees)_sin',
        'Right Ascension of the Ascending Node (degrees)_cos',
        'Argument of Perigee (degrees)_sin',
        'Argument of Perigee (degrees)_cos', 
        'Mean Anomaly (degrees)_sin',
        'Mean Anomaly (degrees)_cos']

In [38]:
all_features = original_features+diff_features+cyclical_features

# Generating lag features

In [28]:
data = data_for_training[all_features].copy()
data.head()

,First Derivative Mean Motion,Inclination (degrees),Eccentricity,Mean Motion (revolutions per day),Revolution Number at Epoch_Diff,Right Ascension of the Ascending Node (degrees)_sin,Right Ascension of the Ascending Node (degrees)_cos,Argument of Perigee (degrees)_sin,Argument of Perigee (degrees)_cos,Mean Anomaly (degrees)_sin,Mean Anomaly (degrees)_cos
Epoch,,,,,,,,,,,
2023-01-01,0.000166,51.644567,0.000519,15.498221,9.166667,0.961985,0.273101,-0.516587,-0.856235,-0.991910,0.126945
2023-01-02,0.000175,51.644680,0.000511,15.498584,14.333333,0.937007,0.349311,-0.583451,-0.812148,-0.664510,-0.747279
2023-01-03,0.000162,51.644860,0.000499,15.498874,15.200000,0.904183,0.427146,-0.640969,-0.767566,-0.384036,-0.923318
2023-01-04,0.000142,51.644740,0.000497,15.499123,17.600000,0.857816,0.513957,-0.715327,-0.698790,-0.636479,-0.771294
2023-01-05,0.000139,51.644820,0.000490,15.499368,13.800000,0.816115,0.577890,-0.754367,-0.656453,-0.944209,-0.329347


In [29]:
look_back = 2 # numbers of lags
forecast_horizon = 1
features = []
targets = []
num_samples = data_for_training.shape[0]

In [30]:
for i in range(num_samples - look_back - forecast_horizon + 1):
    # The input features are the last `look_back` values of all series
    X_window = data.iloc[i : i + look_back].values
    
    # The target variables are the next `forecast_horizon` values of all series
    y_window = data.iloc[i + look_back : i + look_back + forecast_horizon].values
    
    # Flatten the windows to create a single row for the features and targets
    features.append(X_window.flatten())
    targets.append(y_window.flatten())

X = np.array(features)
y = np.array(targets)

print(f"Shape of X (input features): {X.shape}")
print(f"Shape of y (target variables): {y.shape}")

Shape of X (input features): (728, 22)
Shape of y (target variables): (728, 11)


In [31]:
# --- 3. Scaling the Data ---
# It's good practice to scale the data, especially for models that are
# sensitive to feature magnitudes. We use a MinMaxScaler.
# We need to scale X and y separately, and use a different scaler for each
# to allow for inverse transformation later.

# X_scaler = MinMaxScaler()
# y_scaler = MinMaxScaler()

X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_scaled = X_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y)

# --- 4. Splitting the Data ---
# For time series, we split chronologically. The last 20% of the data
# will be used for testing.
split_index = int(len(X_scaled) * 0.8)
X_train, X_test = X_scaled[:split_index], X_scaled[split_index:]
y_train, y_test = y_scaled[:split_index], y_scaled[split_index:]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((582, 22), (582, 11), (146, 22), (146, 11))

# Traing classical ML

In [40]:
# --- Configuration ---
SAVE_DIR = "saved_model_state"
os.makedirs(SAVE_DIR, exist_ok=True)

models_and_params = [

    {
        'name': 'RandomForestRegressor',
        'estimator': RandomForestRegressor(random_state=42, n_jobs=1), # Use n_jobs=1 for inner estimator
        'param_grid': {
            'estimator__n_estimators': [50, 100,],
            'estimator__max_depth': [5, 10, 100]
        }
    },
    # 3. Linear/Robust Models
    {
        'name': 'RidgeRegressor',
        'estimator': Ridge(random_state=42),
        'param_grid': {
            'estimator__alpha': [0.1, 1.0, 10.0]
        }
    },
    {
        'name': 'HuberRegressor',
        'estimator': HuberRegressor(max_iter=1000), # Huber is robust to outliers
        'param_grid': {
            'estimator__epsilon': [1.1, 1.35]
        }
    },
    # 4. Neighbors/SVM
    {
        'name': 'KNeighborsRegressor',
        'estimator': KNeighborsRegressor(n_jobs=1),
        'param_grid': {
            'estimator__n_neighbors': [5, 10],
            'estimator__weights': ['uniform', 'distance']
        }
    },
    # NOTE: SVR (Support Vector Regression) 
    {
        'name': 'SVR',
        'estimator': SVR(),
        'param_grid': {
            'estimator__C': [1.0, 10.0, 0.1, 100.0],
            'estimator__kernel': ['rbf']
        }
    },
    # 5. Simple Neural Network (MLP)
    {
        'name': 'MLPRegressor',
        'estimator': MLPRegressor(random_state=42, max_iter=1000, early_stopping=True),
        'param_grid': {
            'estimator__hidden_layer_sizes': [(50,), (100,),],
            'estimator__activation': ['relu', 'sigmoid']
        }
    }
]


In [ ]:
# --- Training and Saving Loop ---
best_models = {}

print(f"Starting Grid Search CV for {len(models_and_params)} Multi-Output Regressors...")
print(f"Models will be saved to: {SAVE_DIR}")

for item in models_and_params:
    model_name = item['name']
    base_estimator = item['estimator']
    param_grid = item['param_grid']
    
    # 1. Wrap the estimator with MultiOutputRegressor
    # n_jobs=-1 here parallelizes the training across the multiple output targets.
    multi_output_model = MultiOutputRegressor(base_estimator, n_jobs=1)
    
    # 2. Configure GridSearchCV
    # The 'estimator__' prefix targets the inner regressor's parameters.
    grid_search = GridSearchCV(
        estimator=multi_output_model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=3,
        verbose=1, # Reduced verbosity for a cleaner output with many models
        n_jobs=1 # Parallelize the folds/hyperparameters across 1 core to avoid conflicts with MultiOutput's n_jobs=-1
    )
    
    print(f"\n--- Training {model_name} ---")
    
    try:
        # 3. Fit the model
        # X_train and y_train are assumed to be defined and available
        grid_search.fit(X_train, y_train) 
        
        # 4. Save the best estimator
        best_model = grid_search.best_estimator_
        file_path = os.path.join(SAVE_DIR, f'best_{model_name}.joblib')
        joblib.dump(best_model, file_path)
        
        best_models[model_name] = {
            'best_score': grid_search.best_score_,
            'best_params': grid_search.best_params_,
            'file_path': file_path
        }
        
        print(f"Finished {model_name}. Best score: {grid_search.best_score_:.4f}. Model saved to: {file_path}")
        
    except Exception as e:
        print(f"Error training {model_name}: {e}")



Starting Grid Search CV for 6 Multi-Output Regressors...
Models will be saved to: saved_model_state

--- Training RandomForestRegressor ---
Fitting 3 folds for each of 6 candidates, totalling 18 fits


In [ ]:
print("\n--- Summary of Best Models ---")
for name, data in best_models.items():
    print(f"Model: {name}")
    print(f"  Best Score (Neg MSE): {data['best_score']:.4f}")
    print(f"  Best Params: {data['best_params']}")

# Revert cyclical features and a differencing feature

In [ ]:
# --- 6. Prediction and Inverse Scaling ---
# Make predictions on the test set. The predictions will be scaled.
y_pred_scaled = model.predict(X_test)

# Inverse transform the predictions and the actual test data to their original scale
y_pred_original = y_scaler.inverse_transform(y_pred_scaled)
y_test_original = y_scaler.inverse_transform(y_test)


In [ ]:
# --- 7. Evaluation and Visualization ---
# Reshape the data back into its original time series format for plotting and evaluation.
# The shape is (num_test_samples, forecast_horizon * num_series).
# We want to reshape it to (num_test_samples, forecast_horizon, num_series).
y_test_reshaped = y_test_original.reshape(y_test_original.shape[0], forecast_horizon, -1)
y_pred_reshaped = y_pred_original.reshape(y_pred_original.shape[0], forecast_horizon, -1)

In [ ]:
from sklearn.metrics import mean_squared_error


In [ ]:
# Calculate MAE and MAPE for each series over the entire test set
mae_per_series = []
mape_per_series = []
rmse_per_series = []
for i in range(y_test_reshaped.shape[2]):
    actual_series = y_test_reshaped[:, :, i]
    pred_series = y_pred_reshaped[:, :, i]
    
    # Calculate MAE
    mae = mean_absolute_error(actual_series, pred_series)
    mse = mean_squared_error(actual_series, pred_series)
    rmse = np.sqrt(mse)
    rmse_per_series.append(rmse)
    mae_per_series.append(mae)
    
    # Calculate a simple version of MAPE (Mean Absolute Percentage Error)
    # We use a small epsilon to avoid division by zero.
    # Note: MAPE is sensitive to zero or near-zero values.
    non_zero_actuals = actual_series[np.abs(actual_series) > 1e-6]
    if len(non_zero_actuals) > 0:
        percentage_error = np.mean(np.abs((pred_series[np.abs(actual_series) > 1e-6] - non_zero_actuals) / non_zero_actuals)) * 100
    else:
        percentage_error = np.nan
    mape_per_series.append(percentage_error)

print("\nMean Absolute Error (MAE) and Mean Absolute Percentage Error (MAPE) for each time series:")
for i in range(len(mae_per_series)):
#     print(f"  - Series {i+1} ({cols_for_training[i]}): MAE = {mae_per_series[i]:.4f}, MAPE = {mape_per_series[i]:.2f}%")
#     print(f"  - Series {i+1} ({cols_for_training[i]}): MAE = {mae_per_series[i]:.4f}")
    print(f"  - Series {i+1} ({cols_for_training[i]}): RMSE = {rmse_per_series[i]:.4f} :MAE = {mae_per_series[i]:.9f}, MAPE = {mape_per_series[i]:.2f}%")


In [ ]:
y_test_reshaped.shape

In [ ]:
# Plot the predictions vs. actual values for the first time series

for i in range(len(cols_for_training)):
    plt.figure(figsize=(15, 6))
    plt.title(f"Figure {i} {cols_for_training[i]}: Actual vs. Predicted (Test Set)")
    
    if cols_for_training[i] == 'diff_revs_number':
        plt.plot(y_test_reshaped[:, 0, i].cumsum(), label='Actual', color='blue')
        plt.plot(y_pred_reshaped[:, 0, i].cumsum(), label='Predicted', color='red', linestyle='--')

    else:
        plt.plot(y_test_reshaped[:, 0, i], label='Actual', color='blue')
        plt.plot(y_pred_reshaped[:, 0, i], label='Predicted', color='red', linestyle='--')
        
#     plt.plot(y_test_reshaped[:, 0, i], label='Actual', color='blue')
#     plt.plot(y_pred_reshaped[:, 0, i], label='Predicted', color='red', linestyle='--')
    plt.legend()
    plt.grid(True)
    plt.xlabel("Test Sample")
    plt.ylabel(f"{cols_for_training[i]} Value")
    plt.show()

# Hyperparameters tuning

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
# --- 5. Model Training and Hyperparameter Tuning ---
# We will now use GridSearchCV to find the best hyperparameters for multiple models.
# The best model will be selected for evaluation.

models = {
    'RandomForestRegressor': {
        'estimator': RandomForestRegressor(random_state=42),
        'param_grid': {'estimator__n_estimators': [50, 100, 200], 'estimator__max_depth': [None, 5, 10]}
    },
    'KNeighborsRegressor': {
        'estimator': KNeighborsRegressor(),
        'param_grid': {'estimator__n_neighbors': [3, 5, 7], 'estimator__weights': ['uniform', 'distance']}
    },
    'GradientBoostingRegressor': {
        'estimator': GradientBoostingRegressor(random_state=42),
        'param_grid': {'estimator__n_estimators': [50, 100], 'estimator__learning_rate': [0.05, 0.1]}
    },
    # SVR can be very slow, so we use a small, simple grid.
    'SVR': {
        'estimator': SVR(),
        'param_grid': {'estimator__kernel': ['rbf'], 'estimator__C': [0.1, 1, 10]}
    }
}

best_model = None
best_score = -np.inf
best_model_name = ""

print("\nStarting Hyperparameter Tuning...")

for name, model_data in models.items():
    print(f"\n--- Tuning {name} ---")
    base_estimator = model_data['estimator']
    param_grid = model_data['param_grid']
    
    # Wrap the base estimator in MultiOutputRegressor
    multi_output_estimator = MultiOutputRegressor(base_estimator, n_jobs=-1)
    
    # Set up GridSearchCV
    grid_search = GridSearchCV(multi_output_estimator, param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=1)
    
    # Fit the grid search on the training data
    grid_search.fit(X_train, y_train)
    
    print(f"Finished tuning {name}")
    print(f"Best parameters for {name}: {grid_search.best_params_}")
    print(f"Best cross-validation score (negative MAE) for {name}: {grid_search.best_score_:.4f}")
    
    # Check if this is the best model so far
    if grid_search.best_score_ > best_score:
        best_score = grid_search.best_score_
        best_model = grid_search.best_estimator_
        best_model_name = name

print(f"\nModel tuning complete. The overall best model is: {best_model_name}")

In [ ]:
model_filename = 'best_multi_output_model.joblib'
save_path = 'MultiOutputModel'


joblib.dump(best_model, os.path.join(save_path,model_filename))

In [ ]:

# --- 6. Prediction and Inverse Scaling ---
# Make predictions on the test set. The predictions will be scaled.
y_pred_scaled = best_model.predict(X_test)

# Inverse transform the predictions and the actual test data to their original scale
y_pred_original = y_scaler.inverse_transform(y_pred_scaled)
y_test_original = y_scaler.inverse_transform(y_test)

y_test_reshaped = y_test_original.reshape(y_test_original.shape[0], forecast_horizon, -1)
y_pred_reshaped = y_pred_original.reshape(y_pred_original.shape[0], forecast_horizon, -1)



In [ ]:
# Calculate MAE and MAPE for each series over the entire test set
mae_per_series = []
mape_per_series = []
for i in range(y_test_reshaped.shape[2]):
    actual_series = y_test_reshaped[:, :, i]
    pred_series = y_pred_reshaped[:, :, i]
    
    # Calculate MAE
    mae = mean_absolute_error(actual_series, pred_series)
    mae_per_series.append(mae)
    
    # Calculate a simple version of MAPE (Mean Absolute Percentage Error)
    # We use a small epsilon to avoid division by zero.
    # Note: MAPE is sensitive to zero or near-zero values.
    non_zero_actuals = actual_series[np.abs(actual_series) > 1e-6]
    if len(non_zero_actuals) > 0:
        percentage_error = np.mean(np.abs((pred_series[np.abs(actual_series) > 1e-6] - non_zero_actuals) / non_zero_actuals)) * 100
    else:
        percentage_error = np.nan
    mape_per_series.append(percentage_error)

print("\nMean Absolute Error (MAE) and Mean Absolute Percentage Error (MAPE) for each time series:")
for i in range(len(mae_per_series)):
#     print(f"  - Series {i+1} ({cols_for_training[i]}): MAE = {mae_per_series[i]:.4f}, MAPE = {mape_per_series[i]:.2f}%")
    print(f"  - Series {i+1} ({cols_for_training[i]}): MAE = {mae_per_series[i]:.4f}")
